In [ ]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
import requests


spark = SparkSession.builder \
    .appName("DeltaExample") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.2.0") \
    .getOrCreate()

# Obtener datos desde la API
url = "http://host.docker.internal:8000/detalles_compras/"
response = requests.get(url)
if response.status_code != 200:
    raise Exception("Error al conectar con la API")

datos = response.json()

# Crear DataFrame
df = spark.createDataFrame(datos)
df = df.withColumn("precio", df["precio"] * df["cantidad"])

# Guardar como Delta Table
df.write.format("delta").mode("overwrite").save("/home/jovyan/work/detalles_compras_delta")

# Leer la Delta Table y mostrarla
delta_table = DeltaTable.forPath(spark, "/home/jovyan/work/detalles_compras_delta")
delta_table.toDF().show()
